# 😄 表情包生成器 - Qwen Image Edit

基于 Qwen-Image-Edit 模型的智能表情包生成工具

In [ ]:
# 安装必要的依赖
!pip install gradio diffusers modelscope accelerate transformers 

In [ ]:
import gradio as gr
import torch
from PIL import Image
import os
from diffusers import QwenImageEditPipeline
from modelscope import snapshot_download
import numpy as np
import datetime
import glob
from typing import List, Optional

In [ ]:
# 设置设备和数据类型
if torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.bfloat16
    print(f"✅ 使用 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 GPU 内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    device = "cpu"
    torch_dtype = torch.float32
    print("⚠️ 使用 CPU（建议使用GPU以获得更好性能）")

print(f"🔧 设备: {device}, 数据类型: {torch_dtype}")

# 创建输出目录
output_dir = "./emoji_outputs"
os.makedirs(output_dir, exist_ok=True)
print(f"📁 输出目录: {output_dir}")

In [ ]:
# 下载并加载 Qwen-Image-Edit 模型
model_id = "Qwen/Qwen-Image-Edit"
local_dir = './models/Qwen-Image-Edit'

# 检查模型是否已存在
if not os.path.exists(local_dir):
    print(f"📥 开始下载 {model_id} 模型...")
    os.makedirs(os.path.dirname(local_dir), exist_ok=True)
    snapshot_download(model_id, local_dir=local_dir)
    print(f"✅ 模型下载完成: {local_dir}")
else:
    print(f"✅ 模型已存在: {local_dir}")

# 加载管道
print("🔄 正在加载图像编辑管道...")
pipeline = QwenImageEditPipeline.from_pretrained(
    local_dir, 
    torch_dtype=torch_dtype,
    use_safetensors=True,
    device_map="balanced"
)
print("✅ 模型加载完成")

In [ ]:
def generate_emoji(image, prompt, seed=42, steps=30, cfg_scale=4.0):
    """
    生成表情包
    """
    try:
        # 处理输入图像
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        # 编辑参数
        inputs = {
            "image": image,
            "prompt": prompt,
            "generator": torch.manual_seed(seed),
            "true_cfg_scale": cfg_scale,
            "negative_prompt": "blurry, low quality, distorted",
            "num_inference_steps": steps,
        }
        
        # 执行编辑
        with torch.inference_mode():
            output = pipeline(**inputs)
            edited_image = output.images[0]
        
        # 保存生成的图像
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"emoji_{timestamp}_{seed}.png"
        filepath = os.path.join(output_dir, filename)
        edited_image.save(filepath)
        
        return edited_image
        
    except Exception as e:
        print(f"❌ 生成表情包时出错: {str(e)}")
        return image

def get_history_images():
    """
    获取历史生成的图片
    """
    image_files = glob.glob(os.path.join(output_dir, "*.png"))
    image_files.sort(key=os.path.getmtime, reverse=True)  # 按修改时间倒序
    return image_files[:6]  # 返回最新的6张图片

def load_history_image(image_path):
    """
    加载历史图片
    """
    if image_path and os.path.exists(image_path):
        return Image.open(image_path)
    return None

In [ ]:
# 创建表情包生成器界面
def create_emoji_generator():
    # 自定义CSS样式
    custom_css = """
    .gradio-container {
        max-width: 1200px !important;
        margin: auto;
    }
    .image-container {
        border: 2px solid #ddd;
        border-radius: 8px;
        padding: 10px;
    }
    .button-row {
        gap: 8px;
    }
    .history-gallery {
        border: 2px solid #ddd;
        border-radius: 8px;
        padding: 10px;
    }
    """
    
    with gr.Blocks(title="表情包生成器", theme=gr.themes.Soft(), css=custom_css) as demo:
        gr.Markdown(
            """
            # 😄 表情包生成器
            
            使用 Qwen-Image-Edit 模型，将普通图片转换为有趣的表情包！
            
            **使用步骤：** 上传图片 → 选择表情风格或输入自定义提示词 → 生成表情包
            """
        )
        
        with gr.Row():
            # 左侧：输入区域
            with gr.Column(scale=1):
                # 图片输入
                input_image = gr.Image(
                    label="图片输入",
                    type="pil",
                    height=300
                )
                
                # 提示词区域
                gr.Markdown("**提示词**")
                
                # 快速提示词按钮
                with gr.Row():
                    btn_funny = gr.Button("搞笑", size="sm")
                    btn_cute = gr.Button("打哈欠", size="sm")
                    btn_angry = gr.Button("愤怒", size="sm")
                    btn_surprised = gr.Button("惊讶", size="sm")
                
                # 自定义提示词输入
                prompt_input = gr.Textbox(
                    label="",
                    placeholder="输入自定义提示词，例如：让人物做出夸张的表情...",
                    lines=4
                )
                
                # 高级参数
                gr.Markdown("**高级参数**")
                with gr.Accordion("参数设置", open=False):
                    seed_input = gr.Slider(0, 1000, value=42, step=1, label="随机种子")
                    steps_input = gr.Slider(10, 50, value=30, step=5, label="推理步数")
                    cfg_scale_input = gr.Slider(1.0, 8.0, value=4.0, step=0.5, label="CFG缩放")
                
                generate_button = gr.Button("🎨 生成表情包", variant="primary", size="lg")
            
            # 右侧：输出区域
            with gr.Column(scale=1):
                # 图片输出
                output_image = gr.Image(label="图片输出", height=300)
                
                # 历史图片
                gr.Markdown("**历史图片**")
                with gr.Row():
                    history_gallery = gr.Gallery(
                        label="",
                        show_label=False,
                        elem_id="gallery",
                        columns=3,
                        rows=2,
                        height=200,
                        object_fit="contain"
                    )
        
        # 处理函数
        def process_generation(image, prompt, seed, steps, cfg_scale, progress=gr.Progress(track_tqdm=True)):
            if image is None:
                gr.Warning("请先上传图片！")
                return None, get_history_images()
            if not prompt.strip():
                gr.Warning("请输入提示词！")
                return None, get_history_images()
            
            try:
                # 生成表情包
                result_image = generate_emoji(image, prompt, seed, steps, cfg_scale)
                # 更新历史图片
                history_images = get_history_images()
                return result_image, history_images
                
            except Exception as e:
                gr.Error(f"生成失败: {str(e)}")
                return None, get_history_images()
        
        def update_history():
            return get_history_images()
        
        # 绑定事件
        generate_button.click(
            fn=process_generation,
            inputs=[input_image, prompt_input, seed_input, steps_input, cfg_scale_input],
            outputs=[output_image, history_gallery]
        )
        
        # 快速提示词按钮绑定
        emoji_prompts = {
            btn_funny: "Make this person have an exaggerated funny expression with wide eyes and big smile, cartoon style, meme-like",
            btn_cute: "Make this person yawning with sleepy eyes and open mouth, cute and drowsy expression",
            btn_angry: "Make this person have an angry expression with frowning eyebrows and serious face, dramatic lighting",
            btn_surprised: "Make this person have a very surprised expression with wide open eyes and mouth, shocked face"
        }
        
        for btn, prompt in emoji_prompts.items():
            btn.click(fn=lambda x=prompt: x, outputs=[prompt_input])
        
        # 页面加载时更新历史图片
        demo.load(fn=update_history, outputs=[history_gallery])
    
    return demo

In [ ]:
print("🚀 启动表情包生成器...")
    
demo = create_emoji_generator()

# 启动 Gradio 应用
demo.launch(
    share=False,          # 不创建公共链接
    server_name="0.0.0.0",  # 允许外部访问
    server_port=6006,    # 端口号
    show_error=True,     # 显示详细错误信息
    debug=True          # 调试模式
)